In [1]:
import numpy as np
import warnings
import sys
import pandas as pd
import scipy
from scipy.io import arff
if not sys.warnoptions:
    warnings.simplefilter('ignore')
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

C:\Users\User\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\Users\User\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\Users\User\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [2]:
data, meta = scipy.io.arff.loadarff('mtr-datasets/enb.arff')
data = pd.DataFrame(data)
data.head()

,Relative_compactness,X1,X3,X4,X5,X6,X7,X8,Y1,Y2
0,0.98,514.5,294.0,110.25,7.0,2.0,0.0,0.0,15.55,21.33
1,0.98,514.5,294.0,110.25,7.0,3.0,0.0,0.0,15.55,21.33
2,0.98,514.5,294.0,110.25,7.0,4.0,0.0,0.0,15.55,21.33
3,0.98,514.5,294.0,110.25,7.0,5.0,0.0,0.0,15.55,21.33
4,0.90,563.5,318.5,122.50,7.0,2.0,0.0,0.0,20.84,28.28


In [7]:
X = data.iloc[:,:-2].values
y = data.iloc[:,-2:].values
#Making Scaler
x_scaler = StandardScaler()
y_scaler = StandardScaler()

X = x_scaler.fit_transform(X)
y = y_scaler.fit_transform(y)

print(X.shape,y.shape)

#Train-Test Split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=45)

(768, 8) (768, 2)


In [8]:
print("Train_X: ",X_train.shape)
print("Train_Y: ",y_train.shape)
print("Test_X: ",X_test.shape)
print("Test_Y: ",y_test.shape)

Train_X:  (537, 8)
Train_Y:  (537, 2)
Test_X:  (231, 8)
Test_Y:  (231, 2)


In [9]:
from sklearn.metrics import mean_squared_error,mean_absolute_error,f1_score,r2_score
def evaluate(y_train,y_test):
    print("RMSE: ", mean_squared_error(y_train,y_test)**0.5)
    print("MAE: ", mean_absolute_error(y_train,y_test))
    print("R_Squared: ", r2_score(y_train,y_test))
    
    r2 = r2_score(y_train,y_test)
    
    n = 768
    p = 8

    adj_r2 = 1-(1-r2)*(n-1)/(n-p-1)
    print('Adjusted R-squared - ',adj_r2)

### KNN Regression

In [10]:
from sklearn.neighbors import KNeighborsRegressor

In [11]:
for i in range(1,20):
    model = KNeighborsRegressor(n_neighbors=i)
    model.fit(X_train,y_train)
    print(i,":",model.score(X_test,y_test))

1 : 0.9108786505453046
2 : 0.9005876300716197
3 : 0.9166094471406203
4 : 0.9257362636414302
5 : 0.9287495839890547
6 : 0.920424570988809
7 : 0.9214924703032912
8 : 0.9244863126964623
9 : 0.9255298855953125
10 : 0.9259114262713073
11 : 0.920527282854708
12 : 0.9158195579354914
13 : 0.9128271575752065
14 : 0.9138177707036366
15 : 0.9141343315248147
16 : 0.9145564026816339
17 : 0.9110721582073209
18 : 0.9097742609472785
19 : 0.9072009395601215


In [12]:
model = KNeighborsRegressor(n_neighbors=9)
model.fit(X_train,y_train)
evaluate(y_scaler.inverse_transform(y_test),y_scaler.inverse_transform(model.predict(X_test)))

RMSE:  2.6033639032697793
MAE:  1.8166666666666664
R_Squared:  0.9255298855953125
Adjusted R-squared -  0.9247449568532341


### Random Forest

In [13]:
from sklearn.ensemble import RandomForestRegressor

In [14]:
model = RandomForestRegressor(n_estimators=5)
model.fit(X_train,y_train)
evaluate(y_scaler.inverse_transform(y_test),y_scaler.inverse_transform(model.predict(X_test)))

RMSE:  1.5684424264287304
MAE:  0.8380649350649348
R_Squared:  0.9714560975351829
Adjusted R-squared -  0.9711552395381888


### ANN

In [15]:
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


In [18]:
model = Sequential()
model.add(Dense(128,activation='relu',input_dim=X_train.shape[1]))
model.add(Dense(64,activation='relu'))
model.add(Dense(2,activation='linear'))
model.compile(optimizer="adam",loss='mse')

In [19]:
model.fit(X_train,y_train,epochs=500,batch_size=50)


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor

Epoch 1/500
537/537 [==============================] - 0s - loss: 0.6094     
Epoch 2/500
537/537 [==============================] - 0s - loss: 0.2080     
Epoch 3/500
537/537 [==============================] - 0s - loss: 0.1541     
Epoch 4/500
537/537 [==============================] - 0s - loss: 0.1192     
Epoch 5/500
537/537 [==============================] - 0s - loss: 0.1108     
Epoch 6/500
537/537 [==============================] - 0s - loss: 0.1022     
Epoch 7/500
537/537 [==============================] - 0s - loss: 0.0948     
Epoch 8/500
537/537 [==============================] - 0s - loss: 0.0894     
Epoch 9/500
537/537 [==============================] - 0s - loss: 0.0862     
Epoch 10/500
537/537 [==============================] - 0s - loss: 0.0827     
Epoch 11/500
537/537 [==============================] - 0s - loss: 0.0798     
Epoch 12/500
537/537

537/537 [==============================] - 0s - loss: 0.0049     
Epoch 96/500
537/537 [==============================] - 0s - loss: 0.0049     
Epoch 97/500
537/537 [==============================] - 0s - loss: 0.0048     
Epoch 98/500
537/537 [==============================] - 0s - loss: 0.0045     
Epoch 99/500
537/537 [==============================] - 0s - loss: 0.0045     
Epoch 100/500
537/537 [==============================] - 0s - loss: 0.0045     
Epoch 101/500
537/537 [==============================] - 0s - loss: 0.0044     
Epoch 102/500
537/537 [==============================] - 0s - loss: 0.0043     
Epoch 103/500
537/537 [==============================] - 0s - loss: 0.0045     
Epoch 104/500
537/537 [==============================] - 0s - loss: 0.0045     
Epoch 105/500
537/537 [==============================] - 0s - loss: 0.0042     
Epoch 106/500
537/537 [==============================] - 0s - loss: 0.0045     
Epoch 107/500
537/537 [==============================] - 0

537/537 [==============================] - 0s - loss: 0.0018     
Epoch 198/500
537/537 [==============================] - 0s - loss: 0.0017     
Epoch 199/500
537/537 [==============================] - 0s - loss: 0.0016     
Epoch 200/500
537/537 [==============================] - 0s - loss: 0.0017     
Epoch 201/500
537/537 [==============================] - 0s - loss: 0.0017     
Epoch 202/500
537/537 [==============================] - 0s - loss: 0.0016     
Epoch 203/500
537/537 [==============================] - 0s - loss: 0.0017     
Epoch 204/500
537/537 [==============================] - 0s - loss: 0.0016     
Epoch 205/500
537/537 [==============================] - 0s - loss: 0.0017     
Epoch 206/500
537/537 [==============================] - 0s - loss: 0.0017     
Epoch 207/500
537/537 [==============================] - 0s - loss: 0.0018     
Epoch 208/500
537/537 [==============================] - 0s - loss: 0.0015     
Epoch 209/500
537/537 [==============================]

537/537 [==============================] - 0s - loss: 7.7144e-04     
Epoch 299/500
537/537 [==============================] - 0s - loss: 7.3315e-04     
Epoch 300/500
537/537 [==============================] - 0s - loss: 7.4146e-04     
Epoch 301/500
537/537 [==============================] - 0s - loss: 8.3124e-04     
Epoch 302/500
537/537 [==============================] - 0s - loss: 7.5609e-04     
Epoch 303/500
537/537 [==============================] - 0s - loss: 7.2560e-04     
Epoch 304/500
537/537 [==============================] - 0s - loss: 7.0841e-04     
Epoch 305/500
537/537 [==============================] - 0s - loss: 7.1561e-04     
Epoch 306/500
537/537 [==============================] - 0s - loss: 7.5114e-04     
Epoch 307/500
537/537 [==============================] - 0s - loss: 8.1405e-04     
Epoch 308/500
537/537 [==============================] - 0s - loss: 8.1887e-04     
Epoch 309/500
537/537 [==============================] - 0s - loss: 8.5967e-04     
Epoch 

537/537 [==============================] - 0s - loss: 4.5471e-04     
Epoch 397/500
537/537 [==============================] - 0s - loss: 4.8175e-04     
Epoch 398/500
537/537 [==============================] - 0s - loss: 5.8529e-04     
Epoch 399/500
537/537 [==============================] - 0s - loss: 4.9140e-04     
Epoch 400/500
537/537 [==============================] - 0s - loss: 4.7753e-04     
Epoch 401/500
537/537 [==============================] - 0s - loss: 4.1575e-04     
Epoch 402/500
537/537 [==============================] - 0s - loss: 4.2781e-04     
Epoch 403/500
537/537 [==============================] - 0s - loss: 4.5489e-04     
Epoch 404/500
537/537 [==============================] - 0s - loss: 4.2562e-04     
Epoch 405/500
537/537 [==============================] - 0s - loss: 4.5067e-04     
Epoch 406/500
537/537 [==============================] - 0s - loss: 4.5947e-04     
Epoch 407/500
537/537 [==============================] - 0s - loss: 5.3649e-04     
Epoch 

537/537 [==============================] - 0s - loss: 4.6389e-04     
Epoch 494/500
537/537 [==============================] - 0s - loss: 5.2038e-04     
Epoch 495/500
537/537 [==============================] - 0s - loss: 5.3485e-04     
Epoch 496/500
537/537 [==============================] - 0s - loss: 5.5013e-04     
Epoch 497/500
537/537 [==============================] - 0s - loss: 5.2889e-04     
Epoch 498/500
537/537 [==============================] - 0s - loss: 4.5125e-04     
Epoch 499/500
537/537 [==============================] - 0s - loss: 3.7634e-04     
Epoch 500/500
537/537 [==============================] - 0s - loss: 3.4128e-04     


In [20]:
evaluate(y_scaler.inverse_transform(y_test),y_scaler.inverse_transform(model.predict(X_test)))

RMSE:  0.8042394933697522
MAE:  0.5606319031983744
R_Squared:  0.9926894502493362
Adjusted R-squared -  0.9926123957065097
